To complete this challenge, determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

You will need to do considerable data cleaning in order to extract accurate estimates. You may may want to look into data encoding methods if you get stuck. For a real bonus round, identify the open access prices paid by subject area.

Remember not to modify the data directly. Instead, write a cleaning script that will load the raw data and whip it into shape. Jupyter notebooks are a great format for this. Keep a record of your decisions: well-commented code is a must for recording your data cleaning decision-making progress. Submit a link to your script and results below and discuss it with your mentor at your next session.

Data file at https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/WELLCOME/WELLCOME_APCspend2013_forThinkful.csv


In [420]:
#importing modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy


In [268]:
#read data into notebook
url = r"https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/WELLCOME/WELLCOME_APCspend2013_forThinkful.csv"
local = r"C:\Users\Chris\Documents\thinkful\data_sets\WELLCOME_APCspend2013_forThinkful.csv"
data_df_raw = pd.read_csv(url, encoding='latin_1')

data_df_raw.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [450]:
#save copy of data
data_df = data_df_raw
data_df.shape

(2127, 5)

In [451]:
#check data type
data_df.dtypes

PMid     object
pub      object
journ    object
art      object
cost     object
dtype: object

In [452]:
#rename columns
data_df.columns = ['PMid', 'pub', 'journ', 'art', 'cost']
data_df.head()

,PMid,pub,journ,art,cost
0,NaN,Cup,Psychological Medicine,Reduced Parahippocampal Cortical Thickness In ...,£0.00
1,Pmc3679557,Acs,Biomacromolecules,Structural Characterization Of A Model Gram-Ne...,£2381.04
2,23043264 Pmc3506128,Acs,J Med Chem,"Fumaroylamino-4,5-Epoxymorphinans And Related ...",£642.56
3,23438330 Pmc3646402,Acs,J Med Chem,Orvinols With Mixed Kappa/Mu Opioid Receptor A...,£669.64
4,23438216 Pmc3601604,Acs,J Org Chem,Regioselective Opening Of Myo-Inositol Orthoes...,£685.88


In [453]:
#sort by article names to check for near duplicates
data_df.sort_values(by='art').head()

,PMid,pub,journ,art,cost
1231,Pmid:24048963 (Epub Sept 2013),Oxford University Press,Journal Of Infectious Diseases,Persistent Endothelial Activation After Plas...,£2841.60
1729,Pmc3536945,Springer,Brain Topography,"""A Novel Method For Reducing The Effect Of Ton...",£1889.90
1026,NaN,Nature,Biosocieties,"""Creating The 'Ethics Industry': Mary Warnock,...",£1800.00
1287,Pmc3547059,Plos,Plos One,"""Involvement Of Ephb1 Receptors Signalling In ...",£1023.41
1485,3569446,Public Library Of Science,Plos One,"""The Words Will Pass With The Blowing Wind""; S...",£825.68


In [454]:
data_df['journ'].head()

0    Psychological Medicine
1         Biomacromolecules
2                J Med Chem
3                J Med Chem
4                J Org Chem
Name: journ, dtype: object

In [455]:
#why doesn't this work?
data_df = data_df.applymap(str.replace(r'\n', ''))

TypeError: replace() takes at least 2 arguments (1 given)

In [456]:
#remove newline chars
data_df.journ = data_df.journ.str.replace(r'\n', '')
data_df.PMid = data_df.PMid.str.replace(r'\n', '')
data_df.pub = data_df.pub.str.replace(r'\n', '')
data_df.art = data_df.art.str.replace(r'\n', '')
data_df.cost = data_df.cost.str.replace(r'\n', '')

In [457]:
#fix journal names
data_df.journ = data_df.journ.str.title()
data_df.PMid = data_df.PMid.str.title()
data_df.pub = data_df.pub.str.title()
data_df.art = data_df.art.str.title()
data_df.cost = data_df.cost.str.title()

In [458]:
data_df.head()

,PMid,pub,journ,art,cost
0,NaN,Cup,Psychological Medicine,Reduced Parahippocampal Cortical Thickness In ...,£0.00
1,Pmc3679557,Acs,Biomacromolecules,Structural Characterization Of A Model Gram-Ne...,£2381.04
2,23043264 Pmc3506128,Acs,J Med Chem,"Fumaroylamino-4,5-Epoxymorphinans And Related ...",£642.56
3,23438330 Pmc3646402,Acs,J Med Chem,Orvinols With Mixed Kappa/Mu Opioid Receptor A...,£669.64
4,23438216 Pmc3601604,Acs,J Org Chem,Regioselective Opening Of Myo-Inositol Orthoes...,£685.88


In [486]:
#check for near duplicate journal names
data_df['journ'].unique()[:40]

array(['Psychological Medicine', 'Biomacromolecules', 'J Med Chem',
       'J Org Chem', 'Journal Of Medicinal Chemistry',
       'Journal Of Proteome Research', 'Mol Pharm',
       'Acs Chemical Biology',
       'Journal Of Chemical Information And Modeling', 'Biochemistry',
       'Gastroenterology', 'Journal Of Biological Chemistry',
       'Journal Of Immunology', 'Acs Chemical Neuroscience', 'Acs Nano',
       'American Chemical Society', 'Analytical Chemistry',
       'Bioconjugate Chemistry',
       'Journal Of The American Chemical Society', 'Chest',
       'Journal Of Neurophysiology', 'Journal Of Physiology',
       'The Journal Of Neurophysiology', 'American Journal Of Psychiatry',
       'Americal Journal Of Psychiatry', 'Behavioral Neuroscience',
       'Emotion', 'Health Psychology', 'Journal Of Abnormal Psychology',
       'Journal Of Consulting And Clinical Psychology',
       'Journal Of Experimental Psychology:  Animal Behaviour Process',
       'Journal Of Experiment

In [460]:
#remove record with null jornal
data_df = data_df[data_df['journ'].notnull()]


In [461]:
#clean journal names
data_df.loc[:,'journ'] = data_df.loc[:,'journ'].str.replace(r"\s+?", r" ")
data_df.loc[:,'journ'] = data_df.loc[:,'journ'].str.replace(r"\s$", r"")
data_df.loc[:,'journ'] = data_df.loc[:,'journ'].str.replace(r"Public Library Of Science One", r"Plos One")
data_df.loc[:,'journ'] = data_df.loc[:,'journ'].str.replace(r"&", r"and")
data_df.loc[:,'journ'] = data_df.loc[:,'journ'].map(lambda x: x.title())

c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [462]:
#Extract pmids and pmcids
data_df['PMid2'] = data_df.PMid.str.extract(r"((?<!\d)\d{8}(?!\d))")
data_df['PMCid'] = data_df.PMid.str.extract(r"(?<!\d)(\d{7})(?!\d)")


c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [463]:
data_df.head()


,PMid,pub,journ,art,cost,PMid2,PMCid
0,NaN,Cup,Psychological Medicine,Reduced Parahippocampal Cortical Thickness In ...,£0.00,NaN,NaN
1,Pmc3679557,Acs,Biomacromolecules,Structural Characterization Of A Model Gram-Ne...,£2381.04,NaN,3679557
2,23043264 Pmc3506128,Acs,J Med Chem,"Fumaroylamino-4,5-Epoxymorphinans And Related ...",£642.56,23043264,3506128
3,23438330 Pmc3646402,Acs,J Med Chem,Orvinols With Mixed Kappa/Mu Opioid Receptor A...,£669.64,23438330,3646402
4,23438216 Pmc3601604,Acs,J Org Chem,Regioselective Opening Of Myo-Inositol Orthoes...,£685.88,23438216,3601604


In [464]:
#show duplicates on id - not wrokig?
data_df[data_df['PMCid'].duplicated(keep=False)].sort_values(by=['PMCid'], ascending=True).head()


,PMid,pub,journ,art,cost,PMid2,PMCid
128,Pmc3173209,Asbmb,Journal Of Biological Chemistry,The T Cell Receptor Triggering Apparatus Is Co...,£1286.86,NaN,3173209
155,Pmc3173209,Asbmb,The Journal Of Biological Chemistry,The T-Cel Receptor Triggering Apparatus Is Com...,£1281.15,NaN,3173209
176,22738332 Pmc3381227,Biomed Central,Biomed Central,Long-Term Impact Of Systemic Bacterial Infecti...,£1350.00,22738332,3381227
546,22155499 Pmc3381227,Elsevier,Elsevier,Age Related Changes In Microglial Phenotype Va...,£2152.76,22155499,3381227
2063,Pmid: 23344974 Pmc3401426,Wiley-Blackwell,Chembiochem,Using A Fragment-Based Approach To Target Prot...,£2048.93,23344974,3401426


In [465]:
#change cost column to numeric
data_df["cost_n"] = data_df.cost.str.extract(r"£(.*)")
data_df.cost_n = pd.to_numeric(data_df["cost_n"], errors='coerce')
data_df.cost_n.sum()


c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


51159218.16

In [466]:
#remove null costs
data_df[data_df['cost_n'].isnull()]
#cost nulls are actually dollars. 


,PMid,pub,journ,art,cost,PMid2,PMCid,cost_n
178,Pmc3398262,Biomed Central,Bmc Biology,Detailed Interrogation Of Trypanosome Cell Bio...,1674$,NaN,3398262,NaN
179,Pmid: 24020915 Pmc3846689,Biomed Central,Bmc Genetics,The Physical Capability Of Community-Based Men...,1375.8$,24020915,3846689,NaN
180,Pmid:23442822,Biomed Central,Bmc Genome Biology,Patterns Of Prokaryotic Lateral Gene Transfers...,2010$,23442822,NaN,NaN
181,Pmc2843621,Biomed Central,Bmc Genomics,Trichomonas Vaginalis Vast Bspa-Like Gene Fami...,1204.38$,NaN,2843621,NaN
182,Pmcid: Pmc3636053,Biomed Central,Bmc Genomics,Enhancing The Utility Of Proteomics Signature ...,1254.6$,NaN,3636053,NaN
183,3526451,Biomed Central,Bmc Genomics,Advances In Genome-Wide Rnai Cellular Screens:...,1476$,NaN,3526451,NaN
337,23931322 Pmcid: Pmc3736666,Byophysical Society,Biophysical Journal,Aggregation Modulators Interfer With Membrane ...,671.04$,23931322,3736666,NaN
1599,23308065,Public Library Of Science,Plos Pathogens,Transmission Of Equine Influenza Virus During ...,1440$,23308065,NaN,NaN
1600,Pmid:23633946,Public Library Of Science,Plos Pathogens,The Mnn2 Mannosyltransferase Family Modulates ...,1460.3$,23633946,NaN,NaN
1601,Pmc3610638,Public Library Of Science,Plos Pathogens,Dna Break Site At Fragile Subtelomeres Determi...,1476.47$,NaN,3610638,NaN


In [467]:
#convert dollar costs to pounds
data_nonquid = data_df[data_df['cost_n'].isnull()]
data_nonquid.loc[:,'cost_n'] = pd.to_numeric(data_nonquid.loc[:,'cost'].str.replace(r"\$", "", regex=True))
data_df.update(data_nonquid, join='left', overwrite=True, filter_func=None, errors='ignore')
#check for nulls
#data_df[data_df['cost_n'].isnull()]

#this isn't putting updated value back in??

c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\frame.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[col] = expressions.where(mask, this, that)


In [468]:
data_df[data_df['cost_n'].isnull()]
#data_df

,PMid,pub,journ,art,cost,PMid2,PMCid,cost_n


In [469]:
#confirm cost is numeric now
data_df.dtypes

PMid       object
pub        object
journ      object
art        object
cost       object
PMid2      object
PMCid      object
cost_n    float64
dtype: object

In [470]:
#separate nulls and nonnulls
data_df_PMCnotnulls = data_df[pd.isnull(data_df['PMCid'])]
data_df_PMnotnulls = data_df[pd.isnull(data_df['PMid2'])]                           
data_df_nulls = data_df[pd.isnull(data_df['PMid2']) & pd.isnull(data_df['PMCid'])]
data_df[pd.isnull(data_df['PMid2']) & pd.isnull(data_df['PMCid'])].head()

,PMid,pub,journ,art,cost,PMid2,PMCid,cost_n
0,NaN,Cup,Psychological Medicine,Reduced Parahippocampal Cortical Thickness In ...,£0.00,NaN,NaN,0.00
21,NaN,American Chemical Society,Acs Chemical Biology,Discovery Of ?2 Adrenergic Receptor Ligands Us...,£947.07,NaN,NaN,947.07
43,NaN,American Psychiatric Association,American Journal Of Psychiatry,Methamphetamine-Induced Disruption Of Frontost...,£2351.73,NaN,NaN,2351.73
90,Pmc In Progress,American Society For Microbiology,Infection And Immunity,Analysis Of Antibodies To Newly Described Plas...,£2034.00,NaN,NaN,2034.00
93,NaN,American Society For Microbiology,Journal Of Virology,The Human Adenovirus Type 5 L4 Promoter Is Act...,£1312.59,NaN,NaN,1312.59


In [471]:
data_df.shape

(2126, 8)

In [472]:
data_df_PMnotnulls.shape

(1672, 8)

In [473]:
#more PMid2 field is filled out, will be used to aggregate.
data_df_PMCnotnulls.shape

(479, 8)

In [474]:
#fill in missing PMcid from other entries
data_df_nulls.loc[:,'PMCid_assumed'] = data_df_nulls.merge(data_df_PMCnotnulls, on='art').loc[:,'PMCid_y']
data_df_nulls.loc[:,'PMid2_assumed'] = data_df_nulls.merge(data_df_PMnotnulls, on='art').loc[:,'PMid2_y']

c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [475]:
data_df_nulls.head()
#this wasn't able to fill in the nulls.

,PMid,pub,journ,art,cost,PMid2,PMCid,cost_n,PMCid_assumed,PMid2_assumed
0,NaN,Cup,Psychological Medicine,Reduced Parahippocampal Cortical Thickness In ...,£0.00,NaN,NaN,0.00,NaN,NaN
21,NaN,American Chemical Society,Acs Chemical Biology,Discovery Of ?2 Adrenergic Receptor Ligands Us...,£947.07,NaN,NaN,947.07,NaN,NaN
43,NaN,American Psychiatric Association,American Journal Of Psychiatry,Methamphetamine-Induced Disruption Of Frontost...,£2351.73,NaN,NaN,2351.73,NaN,NaN
90,Pmc In Progress,American Society For Microbiology,Infection And Immunity,Analysis Of Antibodies To Newly Described Plas...,£2034.00,NaN,NaN,2034.00,NaN,NaN
93,NaN,American Society For Microbiology,Journal Of Virology,The Human Adenovirus Type 5 L4 Promoter Is Act...,£1312.59,NaN,NaN,1312.59,NaN,NaN


In [476]:
#check for outliers
data_df.sort_values(by='cost_n', ascending=False).head(20)

,PMid,pub,journ,art,cost,PMid2,PMCid,cost_n
1566,Pmcid: Pmc3767740,Public Library Of Science,Plos One,Phenotypic Alteration Of Neutrophils In The Bl...,£999999.00,NaN,3767740,999999.0
410,NaN,Cup,Journal Of The International African Institute,"Pharmacy, Money And Public Health In Dakar.",£999999.00,NaN,NaN,999999.0
1561,23285297,Public Library Of Science,Plos One,Pharmacological Inhibition Of Caspase-2 Protec...,£999999.00,23285297,NaN,999999.0
1562,Pmc3720940,Public Library Of Science,Plos One,Population-Based Cd4 Counts In A Rural Area In...,£999999.00,NaN,3720940,999999.0
1563,Pmcid: Pmc3748079,Public Library Of Science,Plos One,Plant Silac: Stable-Isotope Labelling With Ami...,£999999.00,NaN,3748079,999999.0
825,NaN,Elsevier,Veterinary Parasitology,Persistence Of The Efficacy Of Copper Oxide Wi...,£999999.00,NaN,NaN,999999.0
1208,"Epub Ahead Of Print Pub Jan 2013, Print In Press",Oxford University Press,Cerebral Cortex,Pharmacological Dissociation Of Novelty Respon...,£999999.00,NaN,NaN,999999.0
358,Pmc3219211,Cambridge University Press,Expert Reviews In Molecular Medicine,Pharmacological Targets In The Ubiquitin Syste...,£999999.00,NaN,3219211,999999.0
1046,Pmc3709478,Nature,Nature Communications,Picornavirus Uncoating Intermediate Captured I...,£999999.00,NaN,3709478,999999.0
560,Pmc3632754,Elsevier,Experimental Cell Research,Perturbation Of Invadolysin Disrupts Cell Migr...,£999999.00,NaN,3632754,999999.0


In [427]:
#windsorise cost - super slow?
def windsorise(series, low_lim=.05, up_lim=.95):
    low_lim_n = series.quantile(low_lim) #convert quantile to numeric
    up_lim_n = series.quantile(up_lim) 
    for i, ser in enumerate(series):
        if ser < low_lim_n:
            series[i] = low_lim_n
        if ser > up_lim_n:
            series[i] = up_lim_n
    return series
data_df['cost_n'] = windsorise(data_df['cost_n'])


c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\chris\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

In [485]:
#windsorise with lambda
low = .03
high = .97
quantiles = data_df['cost_n'].quantile([low, high])
data_df['cost_n'] = data_df['cost_n'].apply(lambda x: (max(quantiles[low], x) & (min(quantiles[high], x))))

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [448]:
#summarize dataframe
data_grp_df = data_df_PMnotnulls.groupby('journ').agg({'cost_n': ['sum', 'mean', 'std', 'count']})
data_grp_df.columns = ['_'.join(col).strip() for col in data_grp_df.columns.values]
data_grp_df.columns = ['sum_of_cost', 'mean_cost', 'std_cost', 'article_count']
data_grp_df.sort_values(by='article_count', inplace=True, ascending=False)
data_grp_df.iloc[:8,:]

,sum_of_cost,mean_cost,std_cost,article_count
journ,,,,
Plos One,6338831.10,38185.729518,187409.570898,166
Journal Of Biological Chemistry,1062178.73,23603.971778,148860.768915,45
Neuroimage,55470.60,2218.824000,268.668926,25
Plos Genetics,2030867.15,96707.959524,300303.644375,21
Nucleic Acids Research,20022.00,1053.789474,370.640823,19
Proceedings Of The National Academy Of Sciences,14484.85,804.713889,514.742636,18
Plos Neglected Tropical Diseases,27741.48,1541.193333,146.178543,18
Plos Pathogens,2022219.41,126388.713125,341022.570958,16
